In [61]:
import typing

import weave
from weave.legacy.weave import weave_internal

weave.use_frontend_devmode()
from weave.legacy.weave import ops_domain
from weave.legacy.weave.panels import panel_board

In [62]:
@weave.type()
class Dataset:
    rows: list[typing.Any]

In [64]:
varbar = panel_board.varbar()

entity_name_val = varbar.add("entity_name_val", "shawn", hidden=True)
entity = ops_domain.entity(entity_name_val)
entity_name = varbar.add(
    "entity_name",
    weave.legacy.weave.panels.Dropdown(
        entity_name_val, choices=ops_domain.viewer().entities().name()
    ),
)

project_name_val = varbar.add("project_name_val", "weave-flow1", hidden=True)
project = ops_domain.project(entity_name_val, project_name_val)
project_name = varbar.add(
    "project_name",
    weave.legacy.weave.panels.Dropdown(
        project_name_val, choices=entity.projects().name()
    ),
)

dataset_name_val = varbar.add("dataset_name_val", "my_dataset1", hidden=True)
dataset = varbar.add(
    "dataset",
    weave.legacy.weave.ops.get(
        weave_internal.const("wandb-artifact:///")
        + entity_name_val
        + "/"
        + project_name_val
        + "/"
        + dataset_name_val
        + ":latest/obj"
    ),
    hidden=True,
)
dataset_ref = varbar.add(
    "dataset_ref",
    weave.legacy.weave.ops.ref(
        weave_internal.const("wandb-artifact:///")
        + entity_name_val
        + "/"
        + project_name_val
        + "/"
        + dataset_name_val
        + ":latest/obj"
    ),
)
dataset_name = varbar.add(
    "dataset_name",
    weave.legacy.weave.panels.Dropdown(
        dataset_name_val, choices=project.artifactType("Dataset").artifacts().name()
    ),
)

# Now here I want to pick model A and model B
# I need to a way to get to artifact from an object, like get_ref.artifact ?
# like dataset_ref.artifact().usedBy().filter(lambda row: row.jobName() == 'eval')
# then from there find consuming runs of a given job_type ("eval")
# let the user pick two of those

# want a really nice data grid of the table metric comparsion per row

main = weave.legacy.weave.panels.Group(
    layoutMode="grid",
    showExpressions=True,
    enableAddPanel=True,
)

main.add("ename", entity_name_val)
main.add("pname", project_name_val)
main.add("dsrows", weave.legacy.weave.ops.obj_getattr(dataset, "rows"))
# main.add("artifact", dataset_ref.artifact())

# I Need a programmable Table. One that's columns are defined by Nodes
# Same for panels, I want to be able to generate a bunch of panels
# Can I do this with Weaveify?
#   Like I can pass a function in in the Board that returns the panel I want maybe?

weave.legacy.weave.panels.Board(vars=varbar, panels=main)